In [8]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import csv
from pandas.tseries.offsets import MonthEnd
from datetime import datetime, timedelta, date
import requests
import json 
import ta

In [9]:
#assign dates to dataframe index
NFLX_df = pd.read_csv("/Users/fahad/project_repo/data/final/NFLX/NFLX_normalised.csv", index_col=0)
dates = NFLX_df.index
NFLX_df['Date'] =dates




In [10]:
NFLX_df['Date'] = pd.to_datetime(NFLX_df.Date, format ="%Y-%m-%d")

In [11]:
NFLX_2010 = pd.read_csv("/Users/fahad/project_repo/data/final/NFLX/NFLX_10yr.csv", index_col=0)

In [12]:
#ADD LABELS TO THE TRAINING AND TEST DATAFRAMES AND RETURN DICTIONARIES WITH DATE:LABEL FORMAT
NFLX_2010_labels = labelling(NFLX_2010)

In [13]:
NFLX_2010_flattened= get_ndays_monthly_flattened(NFLX_2010,NFLX_df,30)

In [14]:
NFLX_2010_labels_df = pd.Series(NFLX_2010_labels).to_frame("Label")

In [15]:
#CREATE DATAFRAME FROM FLATTENED DICTIONARY OF ARRAYS FOR TRAINING AND TEST DATA
NFLX_2010_flat_df = pd.DataFrame.from_dict(NFLX_2010_flattened).T

In [16]:
#JOIN THE LABEL DATAFRAME AND THE FLATTENED DATAFRAME ON THE INDEX WHICH IS THE DATE
NFLX_2010_flat_df = NFLX_2010_flat_df.merge(NFLX_2010_labels_df, left_index=True, right_index=True)

In [17]:
NFLX_2010_flat_df

,0,1,2,3,4,5,6,7,8,9,...,351,352,353,354,355,356,357,358,359,Label
2019-07-31,0.430238,0.426036,0.423199,0.418414,0.115910,1.000000,0.249924,0.200136,0.211180,0.125034,...,0.536027,0.059864,0.772802,0.280154,0.449646,0.469981,0.219864,0.0,0.0,1
2019-06-28,0.460354,0.455934,0.459597,0.456598,0.095008,0.744315,0.259892,0.200136,0.211180,0.172472,...,0.501004,0.085541,0.325860,0.216395,0.471042,0.485759,0.197345,0.0,0.0,1
2019-05-31,0.508677,0.513929,0.517315,0.519370,0.065214,0.522492,0.240427,0.449646,0.482062,0.203533,...,0.504122,0.101503,0.546746,0.256692,0.533999,0.524374,0.214815,0.0,0.0,0
2019-04-30,0.490698,0.490505,0.495805,0.493501,0.063158,0.461364,0.234093,0.517165,0.524374,0.228428,...,0.497615,0.284491,0.288726,0.228421,0.523457,0.538595,0.255085,0.0,0.0,1
2019-03-29,0.523571,0.519172,0.525854,0.521946,0.060221,0.516741,0.265624,0.531456,0.531411,0.213650,...,0.510002,0.113372,0.446663,0.252331,0.552357,0.561008,0.251341,0.0,0.0,0
2019-02-28,0.496291,0.492941,0.497104,0.497430,0.068719,0.300248,0.238476,0.536241,0.561008,0.211231,...,0.503780,0.146473,0.447499,0.251348,0.541168,0.561008,0.197088,0.0,0.0,1
2019-01-31,0.509103,0.509959,0.512392,0.509190,0.088715,0.541098,0.252251,0.538465,0.561008,0.218749,...,0.497473,0.333768,0.612087,0.276303,0.537865,0.568690,0.247599,0.0,0.0,0
2018-12-31,0.465400,0.473009,0.470542,0.477569,0.145515,0.308894,0.233645,0.537422,0.567987,0.208783,...,0.366719,0.151821,0.372889,0.256954,0.517607,0.544208,0.348008,0.0,0.0,0
2018-11-30,0.359077,0.360587,0.353563,0.357152,0.173134,0.046560,0.166616,0.526999,0.558501,0.323769,...,0.400789,0.265574,0.589061,0.285140,0.422555,0.468452,0.346026,0.0,0.0,1
2018-10-31,0.393730,0.400811,0.390711,0.403665,0.243166,0.577347,0.309486,0.501049,0.509746,0.324094,...,0.486183,0.290904,0.689754,0.289102,0.401370,0.459277,0.447227,1.0,0.0,1


In [18]:
export = NFLX_2010_flat_df.to_csv("/Users/fahad/project_repo/data/final/NFLX/NFLX_2010_pre_signals.csv")

In [1]:
def labelling(dataframe):
    """This generates labels for the dataframes we wish to predict price.
    A label of 1 suggests an increase in price between months while a label of 0 indicates
    a decrease in price. If the current price is greater than the previous price then a"""
    empty_dict = dict()
    prev_close =0
    counter = 0
    for index,cols in dataframe.iterrows():
        counter +=1
        current_close = cols['Close']
        if float(prev_close) <= float(current_close):
            #signals an increase in price between last month and next month
            empty_dict[index] = 1
        elif float(prev_close) >= float(current_close):
            empty_dict[index]=0

        #reassign prev close to the current close     
        prev_close = current_close
    
    return empty_dict

In [2]:
def get_ndays_monthly_flattened(dataframe_period_end,dataframe_subperiod,n):
    """returns a dictionary which contains key value pair of the month-end date as key and
    value is the proceeding n days. Returns it as a dictionary of dataframes.
    dataframe period end refers to the dataframe which will act as the period end i.e monthly end date
    sub_period dataframe refers to the dataframe with the preceeding n days, i.e daily"""
    empty_dict = dict()
    count = 0
    for end in dataframe_period_end.index:
#         print ("month end/:%s" % end)
        
        month_key = dataframe_period_end.index[count+1]
        
#         print ("month key:%s" % month_key)
        
        last_index = dataframe_period_end.index[-1]
        
#         print ("last index:%s " % last_index)
        
        if (month_key != last_index):
            temp_df = dataframe_subperiod.loc[dataframe_subperiod['Date']<end].head(n)
#             print ("\n")
#             print (temp_df.head(5))
            temp_df = temp_df.drop(['Date'], axis=1)
            
            if (len(temp_df.values.flatten())==360):
                empty_dict[month_key]= temp_df.values.flatten()
                count +=1      
            else:
                pass

        else:
            pass
       
    return empty_dict
    